<a href="https://colab.research.google.com/github/samer-glitch/samerelhajjhassan/blob/main/3%20ensemble%20learning-without%20hyperparameters%20techniques-all%20features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving dataset10allfeatures.csv to dataset10allfeatures.csv


In [2]:
import pandas as pd
import io

# Load the dataset into a pandas dataframe
df = pd.read_csv(io.BytesIO(uploaded['dataset10allfeatures.csv']))
df.head()

,flow_key,src_ip_numeric,src_ip,src_port,dst_ip,dst_port,proto,pktTotalCount,octetTotalCount,min_ps,...,b_flowEnd,b_flowDuration,b_min_piat,b_max_piat,b_avg_piat,b_std_dev_piat,flowEndReason,Networkcategory,application_protocol,web_service
0,3acee4f4ea001cd5e6d9584d4036b53d,3.232266e+09,192.168.121.1,67.0,172.16.255.185,67.0,17.0,22.0,7620.0,328.0,...,1.555955e+09,1.550000e+12,0.010354,198.657965,51.689181,84.916348,2.0,Network,Unknown,DHCP
1,974ec5991b439c9a7176b88be0c90df0,3.232266e+09,192.168.121.1,67.0,172.16.255.186,67.0,17.0,17.0,5670.0,328.0,...,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,2.0,Network,Unknown,DHCP
2,3acee4f4ea001cd5e6d9584d4036b53d,3.232266e+09,192.168.121.1,67.0,172.16.255.185,67.0,17.0,43.0,15124.0,328.0,...,1.555965e+09,1.550000e+12,0.022332,340.268454,89.927588,124.270745,2.0,Network,Unknown,DHCP
3,974ec5991b439c9a7176b88be0c90df0,3.232266e+09,192.168.121.1,67.0,172.16.255.186,67.0,17.0,30.0,10086.0,666666.0,...,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,2.0,Network,Unknown,DHCP
4,cfa7c2740072befaa89c202499729e08,3.232266e+09,192.168.121.1,0.0,10.130.1.166,0.0,1.0,1.0,56.0,56.0,...,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,2.0,Network,Unknown,ICMP


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE

# Load the dataset
df = pd.read_csv('dataset10allfeatures.csv')

# Define the label_traffic function with updated criteria for balanced labeling
def label_traffic(row):
    # Define hypothetical thresholds for normal behavior
    normal_port_range = range(0, 49152)  # Common port range
    normal_protocols = [6, 17]  # TCP and UDP
    normal_pkt_count_range = range(0, 2000)
    normal_octet_count_range = range(0, 1000000)
    normal_packet_size_range = range(20, 1500)
    normal_flow_duration_range = range(50, 3600000)  # 50 ms to 1 hour
    normal_piat_range = range(0, 10000)  # 0 to 10 seconds
    normal_flow_start_end_range = range(0, int(1e12))  # Assuming timestamp in milliseconds

    minor_violations = 0
    major_violations = 0

    # Major violations
    if (row['proto'] not in normal_protocols or
        row['flowStart'] not in normal_flow_start_end_range or
        row['flowEnd'] not in normal_flow_start_end_range or
        row['flowEndReason'] == 1):
        major_violations += 1

    # Minor violations
    if (row['src_port'] not in normal_port_range or
        row['dst_port'] not in normal_port_range or
        row['pktTotalCount'] not in normal_pkt_count_range or
        row['octetTotalCount'] not in normal_octet_count_range or
        row['min_ps'] not in normal_packet_size_range or
        row['max_ps'] not in normal_packet_size_range or
        row['avg_ps'] not in normal_packet_size_range or
        row['std_dev_ps'] not in normal_packet_size_range or
        row['flowDuration'] not in normal_flow_duration_range or
        row['min_piat'] not in normal_piat_range or
        row['max_piat'] not in normal_piat_range or
        row['avg_piat'] not in normal_piat_range or
        row['std_dev_piat'] not in normal_piat_range or
        row['f_pktTotalCount'] not in normal_pkt_count_range or
        row['f_octetTotalCount'] not in normal_octet_count_range or
        row['f_min_ps'] not in normal_packet_size_range or
        row['f_max_ps'] not in normal_packet_size_range or
        row['f_avg_ps'] not in normal_packet_size_range or
        row['f_std_dev_ps'] not in normal_packet_size_range or
        row['f_flowDuration'] not in normal_flow_duration_range or
        row['f_min_piat'] not in normal_piat_range or
        row['f_max_piat'] not in normal_piat_range or
        row['f_avg_piat'] not in normal_piat_range or
        row['f_std_dev_piat'] not in normal_piat_range or
        row['b_pktTotalCount'] not in normal_pkt_count_range or
        row['b_octetTotalCount'] not in normal_octet_count_range or
        row['b_min_ps'] not in normal_packet_size_range or
        row['b_max_ps'] not in normal_packet_size_range or
        row['b_avg_ps'] not in normal_packet_size_range or
        row['b_std_dev_ps'] not in normal_packet_size_range or
        row['b_flowDuration'] not in normal_flow_duration_range or
        row['b_min_piat'] not in normal_piat_range or
        row['b_max_piat'] not in normal_piat_range or
        row['b_avg_piat'] not in normal_piat_range or
        row['b_std_dev_piat'] not in normal_piat_range):
        minor_violations += 1

    # Rule for labeling as 'malicious'
    if major_violations > 0 or minor_violations >= 2:
        return 'malicious'
    else:
        return 'benign'

# Apply the labeling function
df['category'] = df.apply(label_traffic, axis=1)

# Encoding the categorical labels
label_encoder = LabelEncoder()
df['category'] = label_encoder.fit_transform(df['category'])

# Define features and target variable
X = df.drop(columns=['category'])
y = df['category']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Define the preprocessing steps for numeric and categorical columns
numeric_features = X.select_dtypes(include=['number']).columns.tolist()
categorical_features = ['flow_key', 'src_ip', 'dst_ip', 'application_protocol', 'web_service']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Apply preprocessing to training data
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Check the class distribution in the training set
print("Class distribution in training set:")
print(pd.Series(y_train).value_counts())

# Apply SMOTE to the training data
smallest_class_size = pd.Series(y_train).value_counts().min()
k_neighbors = min(smallest_class_size - 1, 5)  # Adjust based on the smallest class size
smote = SMOTE(sampling_strategy='auto', random_state=42, k_neighbors=k_neighbors)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_preprocessed, y_train)

# Define classifiers with default parameters
rf_clf = RandomForestClassifier(random_state=42, n_jobs=-1, class_weight='balanced_subsample')
ab_clf = AdaBoostClassifier(random_state=42)
gb_clf = GradientBoostingClassifier(random_state=42)

# Fit the classifiers to the resampled training data
rf_clf.fit(X_train_resampled, y_train_resampled)
ab_clf.fit(X_train_resampled, y_train_resampled)
gb_clf.fit(X_train_resampled, y_train_resampled)

# Make predictions on the test set
rf_test_predictions = rf_clf.predict(X_test_preprocessed)
ab_test_predictions = ab_clf.predict(X_test_preprocessed)
gb_test_predictions = gb_clf.predict(X_test_preprocessed)

# Calculate accuracy and F1-score for each classifier on the test set
rf_accuracy = accuracy_score(y_test, rf_test_predictions)
rf_f1_score = f1_score(y_test, rf_test_predictions)
ab_accuracy = accuracy_score(y_test, ab_test_predictions)
ab_f1_score = f1_score(y_test, ab_test_predictions)
gb_accuracy = accuracy_score(y_test, gb_test_predictions)
gb_f1_score = f1_score(y_test, gb_test_predictions)

# Print the results
print("Random Forest Classifier:")
print("Accuracy:", rf_accuracy)
print("F1-Score:", rf_f1_score)

print("\nAdaBoost Classifier:")
print("Accuracy:", ab_accuracy)
print("F1-Score:", ab_f1_score)

print("\nGradient Boosting Classifier:")
print("Accuracy:", gb_accuracy)
print("F1-Score:", gb_f1_score)


Class distribution in training set:
1    15
0     6
Name: category, dtype: int64
Random Forest Classifier:
Accuracy: 1.0
F1-Score: 1.0

AdaBoost Classifier:
Accuracy: 1.0
F1-Score: 1.0

Gradient Boosting Classifier:
Accuracy: 1.0
F1-Score: 1.0
